In [ ]:
# Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials
# Valeria Quiroz  - Gabriel Zapata
%%time
!pip install kaggle
api_token = {"username":"gabrielzapata","key":"45d11139cb4e32995898eae373213b5f"}
import json
import zipfile
import os
with open('/content/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!mkdir /root/.kaggle      
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v/content
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ga-customer-revenue-prediction
if not os.path.exists("/content/competitions/ga-customer-revenue-prediction"):
    os.makedirs("/content/competitions/ga-customer-revenue-prediction")
os.chdir('/content/competitions/ga-customer-revenue-prediction')
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

- path is now set to: /content
 85% 52.0M/61.1M [00:00<00:00, 51.6MB/s]
100% 61.1M/61.1M [00:00<00:00, 83.8MB/s]
  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 173MB/s]
 99% 1.10G/1.11G [00:11<00:00, 119MB/s]
100% 1.11G/1.11G [00:12<00:00, 98.9MB/s]
100% 333M/333M [00:03<00:00, 117MB/s]
100% 333M/333M [00:03<00:00, 101MB/s]
 86% 49.0M/56.8M [00:01<00:00, 28.0MB/s]
100% 56.8M/56.8M [00:01<00:00, 47.0MB/s]
 96% 5.00M/5.23M [00:00<00:00, 34.2MB/s]
100% 5.23M/5.23M [00:00<00:00, 33.4MB/s]
CPU times: user 1min 49s, sys: 38.1 s, total: 2min 28s
Wall time: 7min 42s


### **Librerias**

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from pandas import json_normalize
import seaborn as sns 
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
import plotly.graph_objs as go
from plotly import subplots
import plotly.offline as py
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import datetime
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, KFold, RepeatedKFold
from sklearn.pipeline import Pipeline
#from sklearn.metrics import mean_absolute_percentage_error

### **Explicacion del codigo json sacado de Kaggle, para aplanar los datos**

In [ ]:
def load_df(csv_path='/content/competitions/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train_df = load_df()
#test_df = load_df('/content/competitions/ga-customer-revenue-prediction/test.csv')

Loaded train.csv. Shape: (903653, 55)
CPU times: user 2min 13s, sys: 8.88 s, total: 2min 22s
Wall time: 2min 37s


In [ ]:
train_df = train_df.drop( ['sessionId'] + ["trafficSource.campaignCode"], axis=1)
print(train_df.shape)

(903653, 53)


### **Eliminemos columnas constantes e imputar valores cero a NAN**

In [ ]:
print ("Antes de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)
train_df = train_df.loc[:, (train_df != train_df.iloc[0]).any()]
print ("Despues de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)

Antes de eliminar columnas constantes - shape de train y test datasets:  (903653, 53)
Despues de eliminar columnas constantes - shape de train y test datasets:  (903653, 34)


In [ ]:
# Replace NaN values throughout train dataset
train_df.replace(to_replace=np.nan, value=0, inplace=True)

### **Conversion variable categórica a númerica**

In [ ]:
# etiqueta codifica las variables categóricas y convierte las variables numéricas flotantes
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", "device.isMobile",
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
   

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
device.isMobile
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [ ]:
train_df.to_csv('dataset.csv')

In [ ]:
train_df['date'] = train_df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))###Cambia el formato de la fecha

In [ ]:
Y = train_df["totals.transactionRevenue"].astype('float')
X=train_df.drop(["totals.transactionRevenue"],axis=1)##Eliminamos la salida de los datos de las caracteristicas


In [ ]:
##==================###Fold1######============================
Fold1 = train_df[train_df['date']<=datetime.date(2016,11,1)]
train1=Fold1[Fold1['date']<=datetime.date(2016,9,15)]
Val1=Fold1[Fold1['date']>datetime.date(2016,9,15)]

##==================###Fold2######============================
Foldt = train_df[train_df['date']>datetime.date(2016,11,1)]
Fold2 = Foldt[Foldt['date']<=datetime.date(2017,2,1)]
train2=Fold2[Fold2['date']<=datetime.date(2016,12,15)]
Val2=Fold2[Fold2['date']>datetime.date(2016,12,15)]

##==================###Fold3######============================
Foldt2 = train_df[train_df['date']>datetime.date(2017,2,1)]
Fold3 = Foldt2[Foldt2['date']<=datetime.date(2017,5,1)]
train3=Fold3[Fold3['date']<=datetime.date(2017,3,15)]
Val3=Fold3[Fold3['date']>datetime.date(2017,3,15)]

##===================#####All Train#######====================
X_Train=train_df[train_df['date']<=datetime.date(2017,5,1)]
Y_Train=np.log1p(X_Train["totals.transactionRevenue"].astype('float'))

##======================####Test#####==========================
Test = train_df[train_df['date']>datetime.date(2017,5,1)]
y_test= np.log1p(Test["totals.transactionRevenue"].astype('float'))
X_test=Test.drop(["totals.transactionRevenue"],axis=1)

In [ ]:
dev_y1 = np.log1p(train1["totals.transactionRevenue"].astype('float'))
val_y1 = np.log1p(Val1["totals.transactionRevenue"].astype('float'))
dev_y2 = np.log1p(train2["totals.transactionRevenue"].astype('float'))
val_y2 = np.log1p(Val2["totals.transactionRevenue"].astype('float'))
dev_y3 = np.log1p(train3["totals.transactionRevenue"].astype('float'))
val_y3 = np.log1p(Val3["totals.transactionRevenue"].astype('float'))

In [ ]:
X_Train=X_Train.drop(["totals.transactionRevenue"],axis=1)
train1=train1.drop(["totals.transactionRevenue"],axis=1)
train2=train2.drop(["totals.transactionRevenue"],axis=1)
train3=train3.drop(["totals.transactionRevenue"],axis=1)
Val1=Val1.drop(["totals.transactionRevenue"],axis=1)
Val2=Val2.drop(["totals.transactionRevenue"],axis=1)
Val3=Val3.drop(["totals.transactionRevenue"],axis=1)

In [ ]:
X_Train=X_Train.drop(["date"],axis=1)
train1=train1.drop(["date"],axis=1)
train2=train2.drop(["date"],axis=1)
train3=train3.drop(["date"],axis=1)
Val1=Val1.drop(["date"],axis=1)
Val2=Val2.drop(["date"],axis=1)
Val3=Val3.drop(["date"],axis=1)
X_test=X_test.drop(["date"],axis=1)

In [ ]:
n_estimators = [20,30,40,50,60,70,80,90,100]
depths=[3,6,9,12,15,18,21]
EstimadoresError1=[]
for n_estimator in n_estimators:
    for depth in depths:
          clf= GradientBoostingRegressor(n_estimators=n_estimator,max_depth = depth)
          Model1=clf.fit(train1,dev_y1)
          pred_test_y = Model1.predict(Val1)
          pred_test_y[pred_test_y<0]=0
          RMSE=np.sqrt(metrics.mean_squared_error(val_y1,pred_test_y))
          MAE=metrics.mean_absolute_error(val_y1,pred_test_y)
          EstimadoresError1.append([n_estimator,depth,RMSE,MAE])

In [ ]:
n_estimators = [20,30,40,50,60,70,80,90,100]
depths=[3,6,9,12,15,18,21]
EstimadoresError2=[]
for n_estimator in n_estimators:
    for depth in depths:
          clf= GradientBoostingRegressor(n_estimators=n_estimator,max_depth = depth)
          Model2=clf.fit(train2,dev_y2)
          pred_test_y = Model2.predict(Val2)
          pred_test_y[pred_test_y<0]=0
          RMSE=np.sqrt(metrics.mean_squared_error(val_y2,pred_test_y))
          MAE=metrics.mean_absolute_error(val_y2,pred_test_y)
          EstimadoresError2.append([n_estimator,depth,RMSE,MAE])

In [ ]:
n_estimators = [20,30,40,50,60,70,80,90,100]
depths=[3,6,9,12,15,18,21]
EstimadoresError3=[]
for n_estimator in n_estimators:
    for depth in depths:
          clf= GradientBoostingRegressor(n_estimators=n_estimator,max_depth = depth)
          Model3=clf.fit(train3,dev_y3)
          pred_test_y = Model3.predict(Val3)
          pred_test_y[pred_test_y<0]=0
          RMSE=np.sqrt(metrics.mean_squared_error(val_y3,pred_test_y))
          MAE=metrics.mean_absolute_error(val_y3,pred_test_y)
          EstimadoresError3.append([n_estimator,depth,RMSE,MAE])

In [ ]:
dfEstimadoresError=pd.DataFrame({'Modelo1' :EstimadoresError1,'Modelo2' :EstimadoresError2,'Modelo3' :EstimadoresError3})

In [ ]:
dfEstimadoresError.to_csv('EstimadoresError.csv')

In [ ]:
RMSE1=[]
MAE1=[]
n_estimators=[]
depths=[]
l=1
m=1
for i in range(len(EstimadoresError1)):
    mse=(EstimadoresError1[i][2]+EstimadoresError2[i][2]+EstimadoresError3[i][2])/3
    mae=(EstimadoresError1[i][3]+EstimadoresError2[i][3]+EstimadoresError3[i][3])/3
    RMSE1.append(mse)
    MAE1.append(mae)
    n_estimators.append(EstimadoresError1[i][0])
    depths.append(EstimadoresError1[i][1])

In [ ]:
Totaldf1EstimadoresError=pd.DataFrame({'n_estimators': n_estimators,'depths':depths,'RMSE1' :RMSE1,'MAE1' :MAE1,})

In [ ]:
Totaldf1EstimadoresError.to_csv('TotalEstimadoresError.csv')

In [ ]:
Totaldf1EstimadoresError[Totaldf1EstimadoresError['MAE1']==Totaldf1EstimadoresError['MAE1'].min()]

,n_estimators,depths,RMSE1,MAE1
42,80,3,1.639373,0.306337


In [ ]:
Totaldf1EstimadoresError[Totaldf1EstimadoresError['RMSE1']==Totaldf1EstimadoresError['RMSE1'].min()]

,n_estimators,depths,RMSE1,MAE1
49,90,3,1.638708,0.306632


Los mejores parametros calculados para el menor MAE

In [ ]:
BestModelMAE1= GradientBoostingRegressor(n_estimators=80,max_depth =3)

In [ ]:
BestModelMAE1.fit(X_Train,Y_Train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=80,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
pred_test_y1 = BestModelMAE1.predict(X_test)

In [ ]:
#####ERROR POR REGI#########
pred_test_y1[pred_test_y1<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(y_test,pred_test_y1))}")
print(f"MAE= {metrics.mean_absolute_error(y_test,pred_test_y1)}")

RMSE= 1.776833580884741
MAE= 0.3777823033467235


In [ ]:
######ERROR POR USUARIO
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] =(y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y1)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSE
2.1163185788208168
MAE
0.42352173418865596


Error de entrenamiento

In [ ]:
pred_train1_y = BestModelMAE1.predict(train1)
pred_train2_y = BestModelMAE1.predict(train2)
pred_train3_y = BestModelMAE1.predict(train3)

In [ ]:
pred_train1_y[pred_train1_y<0]=0
pred_train2_y[pred_train2_y<0]=0
pred_train3_y[pred_train3_y<0]=0
###ERROR POR REGISTRO#########################
MAEfold1=metrics.mean_absolute_error(dev_y1,pred_train1_y)
MAEfold2=metrics.mean_absolute_error(dev_y2,pred_train2_y)
MAEfold3=metrics.mean_absolute_error(dev_y3,pred_train3_y)
ERROREntremamientaMAE=(MAEfold1+MAEfold2+MAEfold3)/3
print('MAEerrorTrain')
print(ERROREntremamientaMAE)

MAEerrorTrain
0.3140282297232478


In [ ]:
######ERROR POR USUARIO#######################
train1_pred_df = pd.DataFrame({"fullVisitorId":train1["fullVisitorId"].values})
train1_pred_df["transactionRevenue"] =(dev_y1.values)
train1_pred_df["PredictedRevenue"] = (pred_train1_y)
train1_pred_df = train1_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

train2_pred_df = pd.DataFrame({"fullVisitorId":train2["fullVisitorId"].values})
train2_pred_df["transactionRevenue"] =(dev_y2.values)
train2_pred_df["PredictedRevenue"] = (pred_train2_y)
train2_pred_df = train2_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

train3_pred_df = pd.DataFrame({"fullVisitorId":train3["fullVisitorId"].values})
train3_pred_df["transactionRevenue"] =(dev_y3.values)
train3_pred_df["PredictedRevenue"] = (pred_train3_y)
train3_pred_df = train3_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()


MAEfold1U=metrics.mean_absolute_error(train1_pred_df["transactionRevenue"].values, train1_pred_df["PredictedRevenue"].values)
MAEfold2U=metrics.mean_absolute_error(train2_pred_df["transactionRevenue"].values, train2_pred_df["PredictedRevenue"].values)
MAEfold3U=metrics.mean_absolute_error(train3_pred_df["transactionRevenue"].values, train3_pred_df["PredictedRevenue"].values)

ERROREntremamientaMAEU=(MAEfold1U+MAEfold2U+MAEfold3U)/3
print('MAEerrorTrainU')
print(ERROREntremamientaMAEU)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



MAEerrorTrainU
0.32981008194157985


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



#### *Error de Validacion*

In [ ]:
pred_val1_y = BestModelMAE1.predict(Val1)
pred_val2_y = BestModelMAE1.predict(Val2)
pred_val3_y = BestModelMAE1.predict(Val3)

In [ ]:
pred_val1_y[pred_val1_y<0]=0
pred_val2_y[pred_val2_y<0]=0
pred_val3_y[pred_val3_y<0]=0
MAEfoldV1=metrics.mean_absolute_error(val_y1,pred_val1_y)
MAEfoldV2=metrics.mean_absolute_error(val_y2,pred_val2_y)
MAEfoldV3=metrics.mean_absolute_error(val_y3,pred_val3_y)
ERRORValidacionMAEV=(MAEfoldV1+MAEfoldV2+MAEfoldV3)/3
print('MAEerrorVal')
print(ERRORValidacionMAEV)

MAEerrorVal
0.3026348062974284


In [ ]:
######ERROR POR USUARIO#######################
Val1_pred_df = pd.DataFrame({"fullVisitorId":Val1["fullVisitorId"].values})
Val1_pred_df["transactionRevenue"] =(val_y1.values)
Val1_pred_df["PredictedRevenue"] = (pred_val1_y)
Val1_pred_df = Val1_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

Val2_pred_df = pd.DataFrame({"fullVisitorId":Val2["fullVisitorId"].values})
Val2_pred_df["transactionRevenue"] =(val_y2.values)
Val2_pred_df["PredictedRevenue"] = (pred_val2_y)
Val2_pred_df = Val2_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

Val3_pred_df = pd.DataFrame({"fullVisitorId":Val3["fullVisitorId"].values})
Val3_pred_df["transactionRevenue"] =(val_y3.values)
Val3_pred_df["PredictedRevenue"] = (pred_val3_y)
Val3_pred_df = Val3_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

MAEfold1UV=metrics.mean_absolute_error(Val1_pred_df["transactionRevenue"].values, Val1_pred_df["PredictedRevenue"].values)
MAEfold2UV=metrics.mean_absolute_error(Val2_pred_df["transactionRevenue"].values, Val2_pred_df["PredictedRevenue"].values)
MAEfold3UV=metrics.mean_absolute_error(Val3_pred_df["transactionRevenue"].values, Val3_pred_df["PredictedRevenue"].values)

ERRORValidacionMAEU=(MAEfold1UV+MAEfold2UV+MAEfold3UV)/3
print('MAEerrorValU')
print(ERRORValidacionMAEU)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



MAEerrorValU
0.3266194798820837


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Los mejores parametros calculados para el menor MSE

In [ ]:
BestModelMSE1= GradientBoostingRegressor(n_estimators=90,max_depth =3)

In [ ]:
BestModelMSE1.fit(X_Train,Y_Train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=90,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
pred_test_y2 = BestModelMSE1.predict(X_test)


In [ ]:
pred_test_y2[pred_test_y2<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(y_test,pred_test_y2))}")
print(f"MAE= {metrics.mean_absolute_error(y_test,pred_test_y2)}")

RMSE= 1.7769692265374548
MAE= 0.3822864836607959


In [ ]:
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] =  (y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y2)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSE
2.120910151152011
MAE
0.4283524611224513


 ****Error de entrenamiento****


In [ ]:
pred_train1a_y = BestModelMSE1.predict(train1)
pred_train2a_y = BestModelMSE1.predict(train2)
pred_train3a_y = BestModelMSE1.predict(train3)

In [ ]:
pred_train1a_y[pred_train1a_y<0]=0
pred_train2a_y[pred_train2a_y<0]=0
pred_train3a_y[pred_train3a_y<0]=0
###ERROR POR REGISTRO#########################
RMSEfold1=np.sqrt(metrics.mean_squared_error(dev_y1,pred_train1a_y))
RMSEfold2=np.sqrt(metrics.mean_squared_error(dev_y2,pred_train2a_y))
RMSEfold3=np.sqrt(metrics.mean_squared_error(dev_y3,pred_train3a_y))
ERROREntremamientaRMSE=(RMSEfold1+RMSEfold2+RMSEfold3)/3
print('RMSEerrorTrain')
print(ERROREntremamientaRMSE)

RMSEerrorTrain
1.6348661562166444


In [ ]:
######ERROR POR USUARIO#######################
train1a_pred_df = pd.DataFrame({"fullVisitorId":train1["fullVisitorId"].values})
train1a_pred_df["transactionRevenue"] =(dev_y1.values)
train1a_pred_df["PredictedRevenue"] = (pred_train1a_y)
train1a_pred_df = train1a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

train2a_pred_df = pd.DataFrame({"fullVisitorId":train2["fullVisitorId"].values})
train2a_pred_df["transactionRevenue"] =(dev_y2.values)
train2a_pred_df["PredictedRevenue"] = (pred_train2a_y)
train2a_pred_df = train2a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

train3a_pred_df = pd.DataFrame({"fullVisitorId":train3["fullVisitorId"].values})
train3a_pred_df["transactionRevenue"] =(dev_y3.values)
train3a_pred_df["PredictedRevenue"] = (pred_train3a_y)
train3a_pred_df = train3a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()


RMSEfold1UT=np.sqrt(metrics.mean_squared_error(train1a_pred_df["transactionRevenue"].values, train1a_pred_df["PredictedRevenue"].values))
RMSEfold2UT=np.sqrt(metrics.mean_squared_error(train2a_pred_df["transactionRevenue"].values, train2a_pred_df["PredictedRevenue"].values))
RMSEfold3UT=np.sqrt(metrics.mean_squared_error(train3a_pred_df["transactionRevenue"].values, train3a_pred_df["PredictedRevenue"].values))

ERROREntrenamientoRMSEU=(RMSEfold1UT+RMSEfold2UT+RMSEfold3UT)/3
print('RMSEerrorTrainU')
print(ERROREntrenamientoRMSEU)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSEerrorTrainU
1.8998096280067607


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



#### *Error de Validacion*

In [ ]:
pred_val1a_y = BestModelMSE1.predict(Val1)
pred_val2a_y = BestModelMSE1.predict(Val2)
pred_val3a_y = BestModelMSE1.predict(Val3)

In [ ]:
pred_val1a_y[pred_val1a_y<0]=0
pred_val2a_y[pred_val2a_y<0]=0
pred_val3a_y[pred_val3a_y<0]=0
###ERROR POR REGISTRO#########################
RMSEfoldV1=np.sqrt(metrics.mean_squared_error(val_y1,pred_val1a_y))
RMSEfoldV2=np.sqrt(metrics.mean_squared_error(val_y2,pred_val2a_y))
RMSEfoldV3=np.sqrt(metrics.mean_squared_error(val_y3,pred_val3_y))
ERRORValidacionRMSEV=(RMSEfoldV1+RMSEfoldV2+RMSEfoldV3)/3
print('RMSEerrorVal')
print(ERRORValidacionRMSEV)

RMSEerrorVal
1.6097843348865064


In [ ]:
######ERROR POR USUARIO#######################
Val1a_pred_df = pd.DataFrame({"fullVisitorId":Val1["fullVisitorId"].values})
Val1a_pred_df["transactionRevenue"] =(val_y1.values)
Val1a_pred_df["PredictedRevenue"] = (pred_val1a_y)
Val1a_pred_df = Val1a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

Val2a_pred_df = pd.DataFrame({"fullVisitorId":Val2["fullVisitorId"].values})
Val2a_pred_df["transactionRevenue"] =(val_y2.values)
Val2a_pred_df["PredictedRevenue"] = (pred_val2a_y)
Val2a_pred_df = Val2a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

Val3a_pred_df = pd.DataFrame({"fullVisitorId":Val3["fullVisitorId"].values})
Val3a_pred_df["transactionRevenue"] =(val_y3.values)
Val3a_pred_df["PredictedRevenue"] = (pred_val3_y)
Val3a_pred_df = Val3a_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()


RMSEfold1UV=np.sqrt(metrics.mean_squared_error(Val1a_pred_df["transactionRevenue"].values, Val1a_pred_df["PredictedRevenue"].values))
RMSEfold2UV=np.sqrt(metrics.mean_squared_error(Val2a_pred_df["transactionRevenue"].values, Val2a_pred_df["PredictedRevenue"].values))
RMSEfold3UV=np.sqrt(metrics.mean_squared_error(Val3a_pred_df["transactionRevenue"].values, Val3a_pred_df["PredictedRevenue"].values))

ERRORValidacionRMSEU=(RMSEfold1UV+RMSEfold2UV+RMSEfold3UV)/3
print('RMSEerrorValU')
print(ERRORValidacionRMSEU)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSEerrorValU
1.8401498728742272


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

